In [8]:
import os;
import json;
import builtins
import duckdb
import pandas as pd;
import matplotlib.pyplot as plt

def get_dataset_from_testcase(run):
    return run.split("_")[0]
def get_ratio_from_testcase(run):
    return run.split("_")[-1]

dirs = [
    '/home/chesetti/Repos/learned_merge_cleanup/sponge/join_unsorted',
]
runs = []
for dir in dirs:
 for test_case in os.listdir(dir):
    if test_case == 'build':
        continue
    for run_name in os.listdir(os.path.join(dir, test_case, 'outputs', 'results')):
        if (run_name=="run"):
           continue
        json_file = builtins.open(os.path.join(dir, test_case, 'outputs', 'results',  run_name))
        run = (json.load(json_file))
        run['dataset'] = get_dataset_from_testcase(test_case)
        run['ratio'] = get_ratio_from_testcase(run_name)
        runs.append(run)
df = pd.json_normalize(runs)
print(df.columns)
display(df)

Index(['dataset', 'ratio', 'result.bucket_size', 'result.checksum',
       'result.disk_read', 'result.disk_write', 'result.duration_ns',
       'result.duration_sec', 'result.num_output_keys', 'spec.algo',
       'spec.algo_name', 'spec.check_checksum', 'spec.inner_index_file',
       'spec.inner_table', 'spec.key_size', 'spec.key_type',
       'spec.load_sstable_in_mem', 'spec.name', 'spec.num_threads',
       'spec.outer_index_file', 'spec.outer_table', 'spec.result_path',
       'spec.value_size', 'spec.write_result_to_disk'],
      dtype='object')


,dataset,ratio,result.bucket_size,result.checksum,result.disk_read,result.disk_write,result.duration_ns,result.duration_sec,result.num_output_keys,spec.algo,...,spec.key_size,spec.key_type,spec.load_sstable_in_mem,spec.name,spec.num_threads,spec.outer_index_file,spec.outer_table,spec.result_path,spec.value_size,spec.write_result_to_disk
0,fb,1,2074,0,843859892032,6400000000,856580124154,856.580124,200000000,unsorted_lsj_sorted_output,...,8,uint64,False,unsorted_lsj_sorted_output_run0,1,sponge/join_unsorted/fb_threads=1/outputs/unso...,sponge/join_unsorted/fb_threads=1/inputs/table1,sponge/join_unsorted/fb_threads=1/outputs/unso...,8,True
1,fb,100,256,0,847054088112,835257812992,570488613812,570.488614,2000000,unsorted_inlj_sorted_output,...,8,uint64,False,unsorted_inlj_sorted_output_run0,1,sponge/join_unsorted/fb_threads=1/outputs/unso...,sponge/join_unsorted/fb_threads=1/inputs/table100,sponge/join_unsorted/fb_threads=1/outputs/unso...,8,True
2,fb,100,2074,0,20385362528,3232000000,320163374181,320.163374,2000000,unsorted_lsj_sorted_output,...,8,uint64,False,unsorted_lsj_sorted_output_run0,1,sponge/join_unsorted/fb_threads=1/outputs/unso...,sponge/join_unsorted/fb_threads=1/inputs/table100,sponge/join_unsorted/fb_threads=1/outputs/unso...,8,True
3,fb,1000,2074,0,10925892096,3203200000,305173124493,305.173124,200000,unsorted_lsj_sorted_output,...,8,uint64,False,unsorted_lsj_sorted_output_run0,1,sponge/join_unsorted/fb_threads=1/outputs/unso...,sponge/join_unsorted/fb_threads=1/inputs/table...,sponge/join_unsorted/fb_threads=1/outputs/unso...,8,True
4,fb,sanity,934,0,842351760,6400000,484552117,0.484552,200000,unsorted_lsj_sorted_output,...,8,uint64,False,unsorted_lsj_sorted_output_run0,1,sponge/join_unsorted/fb_threads=1/outputs/unso...,sponge/join_unsorted/fb_threads=1/inputs/table...,sponge/join_unsorted/fb_threads=1/outputs/unso...,8,True
5,fb,10,256,0,1068949585008,909697458176,644454610226,644.454610,20000000,unsorted_inlj_sorted_output,...,8,uint64,False,unsorted_inlj_sorted_output_run0,1,sponge/join_unsorted/fb_threads=1/outputs/unso...,sponge/join_unsorted/fb_threads=1/inputs/table10,sponge/join_unsorted/fb_threads=1/outputs/unso...,8,True
6,fb,1,256,0,3287786268656,1653975425024,1490016058612,1490.016059,200000000,unsorted_inlj_sorted_output,...,8,uint64,False,unsorted_inlj_sorted_output_run0,1,sponge/join_unsorted/fb_threads=1/outputs/unso...,sponge/join_unsorted/fb_threads=1/inputs/table1,sponge/join_unsorted/fb_threads=1/outputs/unso...,8,True
7,fb,10,2074,0,95482479040,3520000000,373117111494,373.117111,20000000,unsorted_lsj_sorted_output,...,8,uint64,False,unsorted_lsj_sorted_output_run0,1,sponge/join_unsorted/fb_threads=1/outputs/unso...,sponge/join_unsorted/fb_threads=1/inputs/table10,sponge/join_unsorted/fb_threads=1/outputs/unso...,8,True
8,fb,1000,256,0,824865173456,827814482944,557458684337,557.458684,200000,unsorted_inlj_sorted_output,...,8,uint64,False,unsorted_inlj_sorted_output_run0,1,sponge/join_unsorted/fb_threads=1/outputs/unso...,sponge/join_unsorted/fb_threads=1/inputs/table...,sponge/join_unsorted/fb_threads=1/outputs/unso...,8,True
9,fb,sanity,256,0,2506232464,1653721088,3178866095,3.178866,71161,unsorted_inlj_sorted_output,...,8,uint64,False,unsorted_inlj_sorted_output_run0,1,sponge/join_unsorted/fb_threads=1/outputs/unso...,sponge/join_unsorted/fb_threads=1/inputs/table...,sponge/join_unsorted/fb_threads=1/outputs/unso...,8,True


In [19]:

epsilon = {
    "pgm256": 256,
    "pgm1024": 1024,
    "pgm2048": 2048,
    "flatpgm256": 256,
    "flatpgm1024": 1024,
    "flatpgm2048": 2048,
    "flatpgm4096": 4096,
    "flatpgm8192": 8192,
    "hashjoin": 0,
    "btree": 0,
    "sj": 0,
    "sj2": 0,
    "standard_merge": 0
};
def indexType(x):
    if "pgm" in x:
        return "PGM"
    elif "btree" in x:
        return "BTREE"
    return "NA"

df["algo"] = df["spec.algo"]
df["threads"] = df["spec.num_threads"]
df["disk_read"] = df["result.disk_read"]
df["disk_write"] = df["result.disk_write"]
df["duration_sec"] = df["result.duration_ns"] / (1000000000)
df['dataset'].unique()
df['result.num_output_keys'].unique()
duckdb.sql("SELECT * FROM df WHERE dataset='fb'")


┌─────────┬─────────┬────────────────────┬─────────────────┬───┬──────────────────────┬───────────────┬───────────────┐
│ dataset │  ratio  │ result.bucket_size │ result.checksum │ … │        algos         │   disk_read   │  disk_write   │
│ varchar │ varchar │       int64        │      int64      │   │       varchar        │     int64     │     int64     │
├─────────┼─────────┼────────────────────┼─────────────────┼───┼──────────────────────┼───────────────┼───────────────┤
│ fb      │ 1       │               2074 │               0 │ … │ unsorted_lsj_sorte…  │  843859892032 │    6400000000 │
│ fb      │ 100     │                256 │               0 │ … │ unsorted_inlj_sort…  │  847054088112 │  835257812992 │
│ fb      │ 100     │               2074 │               0 │ … │ unsorted_lsj_sorte…  │   20385362528 │    3232000000 │
│ fb      │ 1000    │               2074 │               0 │ … │ unsorted_lsj_sorte…  │   10925892096 │    3203200000 │
│ fb      │ sanity  │                934

In [23]:
def print_for_dataset_duration(dataset ):
    print("## " + dataset)
    rows = duckdb.sql(
        " SELECT ratio, algo, MEDIAN(duration_sec) as v FROM df"
        f" WHERE threads=1  AND dataset='{dataset}'"
        " GROUP BY dataset, threads, ratio, algo"
        " ORDER BY ratio, threads"
    ).df()
    rows = rows.pivot(index='ratio', values='v', columns='algo')
    display(rows)
    rows.to_csv('unsorted_csv')

def print_for_dataset_bytes_read(dataset ):
    print("## " + dataset + " bytes read")
    rows = duckdb.sql(
        " SELECT ratio, algo, MEDIAN(disk_read) as v FROM df"
        f" WHERE threads=1  AND dataset='{dataset}'"
        " GROUP BY dataset, threads, ratio, algo"
        " ORDER BY ratio, threads"
    ).df()
    rows['v'] = rows['v'] / 4096
    rows = rows.pivot(index='ratio', values='v', columns='algo')
    display(rows)
    rows.to_csv('unsorted_csv')

def print_for_dataset_bytes_written(dataset ):
    print("## " + dataset + " bytes write")
    rows = duckdb.sql(
        " SELECT ratio, algo, MEDIAN(disk_write) as v FROM df"
        f" WHERE threads=1  AND dataset='{dataset}'"
        " GROUP BY dataset, threads, ratio, algo"
        " ORDER BY ratio, threads"
    ).df()
    rows['v'] = rows['v'] / 1e9
    rows = rows.pivot(index='ratio', values='v', columns='algo')
    display(rows)
    rows.to_csv('unsorted_csv')



print_for_dataset_duration("fb")
print_for_dataset_bytes_read("fb")
print_for_dataset_bytes_written("fb")



## fb


algo,unsorted_inlj_sorted_output,unsorted_lsj_sorted_output
ratio,,
1,1490.016059,856.580124
10,644.454610,373.117111
100,570.488614,320.163374
1000,557.458684,305.173124
sanity,3.178866,0.484552


## fb


algo,unsorted_inlj_sorted_output,unsorted_lsj_sorted_output
ratio,,
1,3287.786269,843.859892
10,1068.949585,95.482479
100,847.054088,20.385363
1000,824.865173,10.925892
sanity,2.506232,0.842352


## fb


algo,unsorted_inlj_sorted_output,unsorted_lsj_sorted_output
ratio,,
1,1653.975425,6.4000
10,909.697458,3.5200
100,835.257813,3.2320
1000,827.814483,3.2032
sanity,1.653721,0.0064


In [34]:
algos = ['lsj_sampledflatpgm256', 'lsj_flatpgm256', 'lsj_radixspline256', 'lsj_rmi' , 'lsj_btree256', 'sort_join']
print_for_dataset("fb", algos)
print_for_dataset("wiki", algos)
print_for_dataset("osm", algos)
print_for_dataset("books", algos)


## fb
|   ratio |   lsj_sampledflatpgm256 |   lsj_flatpgm256 |   lsj_radixspline256 |   lsj_rmi |   lsj_btree256 |   sort_join |
|--------:|------------------------:|-----------------:|---------------------:|----------:|---------------:|------------:|
|       1 |               23.2294   |        22.4319   |            25.6702   | 20.3287   |       42.3009  |    11.7042  |
|      10 |                3.68178  |         3.45073  |             4.30738  |  3.76312  |        5.83421 |     4.45029 |
|     100 |                0.98362  |         1.01192  |             1.46279  |  1.25237  |        1.57196 |     3.12833 |
|    1000 |                0.385151 |         0.353513 |             0.374326 |  0.275802 |        0.31908 |     2.86744 |
## wiki
|   ratio |   lsj_sampledflatpgm256 |   lsj_flatpgm256 |   lsj_radixspline256 |   lsj_btree256 |   sort_join |
|--------:|------------------------:|-----------------:|---------------------:|---------------:|------------:|
|       1 |               

In [29]:
algos = ['lsj_radixspline256', 'lsj_radixspline1024', 'lsj_radixspline4096']
print_for_dataset("fb", algos)
print_for_dataset("wiki", algos)
print_for_dataset("osm", algos)
print_for_dataset("books", algos)

## fb


BinderException: Binder Error: Referenced column "threads" not found in FROM clause!
Candidate bindings: "df.dataset"